In [12]:
from mqt.bench import get_benchmark, TKETSettings, CompilerSettings
from typing import Tuple
import cudaq
import time
import re
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm

In [13]:
# cudaq.set_target("equal1", emulate=True, url="http://host.docker.internal:62123")
# cudaq.set_target("nvidia", option="fp64")

In [ ]:
compiler_settings = CompilerSettings(tket=TKETSettings(placement="linplacement"))

algorithm = get_benchmark(
    benchmark_name="random",
    level="nativegates",
    circuit_size=5,
    compiler="tket",
    provider_name="ibm",
    compiler_settings=compiler_settings,
)

257

In [4]:
def benchmark_algorithm(algorithm) -> Tuple[str, float]:
    kernel = cudaq.make_kernel()

    # Allocate the qubits
    for q_register in algorithm.q_registers:
        code = f"{q_register.name} = kernel.qalloc({q_register.size})"
        exec(code)

    # Translate tket commands to cudaq commands
    for command in algorithm.get_commands():
        if command.op.get_name() == "Barrier":
            continue
        elif command.op.get_name() == "Measure":
            assert len(command.qubits) == 1
            assert len(command.bits) == 1
            code = f"kernel.mz({command.qubits[0]}, '{command.bits[0]}')"
        else:
            op_name = command.op.get_name().lower()
            if len(command.op.params) > 0:
                op_name = op_name.split("(")[0]

            if op_name == "sx":
                code = f"kernel.rx(np.pi/2, {','.join([str(q_register) for q_register in command.qubits])})"
            elif op_name == "u1":
                code = re.sub(r"u1\(", "u3(0.0,0.0,", code)
            elif op_name == "u2":
                code = re.sub(r"u2\(", "u3(0.5,", code)
            elif op_name == "ccx":
                code = re.sub(
                    r"ccx\(([^)]+)\)",
                    lambda m: f"cx([{','.join(m.group(1).split(',')[:-1])}],{m.group(1).split(',')[-1]})",
                    code,
                )
            else:
                code = f"kernel.{op_name}({','.join([str(param) for param in command.op.params] + [str(q_register) for q_register in command.qubits])})"

        exec(code)

    # Sample the kernel
    start = time.perf_counter()
    counts = cudaq.sample(kernel)
    end = time.perf_counter()

    return counts, end - start

In [5]:
counts, sample_time = benchmark_algorithm(algorithm)
print(counts)
print(f"Time: {sample_time:.2f} s")

{ 
  __global__ : { 00000:87 00001:73 00010:23 00011:129 00100:17 00101:42 00110:27 00111:1 01000:8 01001:6 01010:27 01011:2 01100:34 01101:60 01110:33 01111:10 10000:38 10001:28 10010:1 10011:52 10100:64 10101:14 10110:1 10111:50 11000:27 11001:2 11010:7 11100:5 11101:59 11110:25 11111:48 }
   meas[0] : { 0:579 1:421 }
   meas[1] : { 0:647 1:353 }
   meas[2] : { 0:564 1:436 }
   meas[3] : { 0:510 1:490 }
   meas[4] : { 0:424 1:576 }
}

Time: 0.13 s


In [6]:
def sample_result_to_dict(counts) -> dict:
    counts_str = str(counts)
    counts_str = re.sub(r"(\S+)\s?:", r'"\1":', counts_str)
    counts_str = counts_str.replace("}", "},")
    counts_str = re.sub(r"(\S+:\S+)", r"\1,", counts_str)
    counts_str = counts_str[:-2]

    counts_dict = ast.literal_eval(counts_str)

    return counts_dict

In [7]:
benchmarks = ["random"]
min_qubits, max_qubits = 2, 16

In [ ]:
def run_benchmarks(benchmarks, min_qubits, max_qubits) -> pd.DataFrame:
    i = 0
    df = pd.DataFrame(
        columns=["benchmark", "qubits", "gates", "depth", "sample_time", "counts"],
    )
    for benchmark_name in benchmarks:
        for qubits in tqdm(range(min_qubits, max_qubits + 1), desc=benchmark_name):
            algorithm = get_benchmark(
                benchmark_name=benchmark_name,
                level="nativegates",
                circuit_size=qubits,
                compiler="tket",
                provider_name="ibm",
                compiler_settings=compiler_settings,
            )

            counts, sample_time = benchmark_algorithm(algorithm)
            counts_dict = sample_result_to_dict(counts)

            df.loc[i] = [
                algorithm.name,
                algorithm.n_qubits,
                algorithm.n_gates,
                algorithm.depth(),
                sample_time,
                counts_dict,
            ]
            i += 1

    return df

In [9]:
df = run_benchmarks(benchmarks, min_qubits, max_qubits)

random:   0%|          | 0/15 [00:00<?, ?it/s]

random: 100%|██████████| 15/15 [01:15<00:00,  5.07s/it]


In [10]:
df.head()

,benchmark,qubits,gates,sample_time,counts
0,random,2,31,0.028124,"{'__global__': {'00': 481, '01': 432, '10': 69..."
1,random,3,260,0.052099,"{'__global__': {'000': 144, '001': 159, '010':..."
2,random,4,437,0.054475,"{'__global__': {'0000': 97, '0001': 218, '0010..."
3,random,5,597,0.066373,"{'__global__': {'00000': 71, '00001': 80, '000..."
4,random,6,985,0.103318,"{'__global__': {'000000': 22, '000001': 22, '0..."


In [11]:
df.to_csv("benchmark_results.csv", index=False)